In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
i = 0

In [ ]:
from image_generation import NutritionLabelGenerator
from augmentation import *

working_dir = "ntbk_test"

generator = NutritionLabelGenerator(output_dir=working_dir)

label_true = generator.generate_nutrition_data()
img_name = f"label_{i:04d}.png"
img_path = generator.generate_image(label_true, img_name)

augment = AugmentationParams.random()
img = cv2.imread(img_path)
img = apply_augmentations(img, augment)
cv2.imwrite(img_path, img)

i = i + 1


In [ ]:
from augmentation import *
from models import *
from deskewing_advanced import DeskewingPipeline
import os
import easyocr

deskewer = DeskewingPipeline(debug=True, debug_dir=working_dir)
img_corrected, rot_est = deskewer.deskew(img, method="hough_lines")

print("rotation:", rot_est)

working_dir = "ntbk_test"
cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_deskewed.png"), img_corrected)

# Run OCR
reader = easyocr.Reader(['en'], gpu=False)
ocr_results = reader.readtext(img_corrected)

bbox_img = draw_bounding_boxes(img_corrected, ocr_results)

cv2.imwrite(os.path.join(working_dir, f"label_{i:04d}_boxes.png"), bbox_img)

import regex_matching as rem

rem.pretty_print_easyocr(ocr_results)

text_in_lines = rem.easyocr_to_lines(ocr_results)
label_pred = rem.extract_nutrients(text_in_lines, rem.nutrient_aliases)

print(label_true)
print(label_pred)


Using CPU. Note: This module is much faster with a GPU.


[[[-487.           3.0019662]]

 [[-484.           3.0019662]]

 [[-492.           3.0194197]]]
rotation: RotationResult(angle=0.0, confidence=0.0, method='hough_lines')
Index  Bounding Box (x,y)                            Text                 Confidence
-------------------------------------------------------------------------------------
0      (296,260), (324,260), (324,286), (296,286)    60}                  0.118877
1      (218,290), (248,290), (248,320), (218,320)    Fet                  0.031452
2      (252,360), (284,360), (284,392), (252,392)    92                   0.296708
3      (249,451), (267,451), (267,473), (249,473)    9                    0.531343
4      (206,146), (277,194), (264,214), (193,165)    utriant              0.618629
5      (173,162), (234,204), (219,226), (158,184)    Calores              0.391357
6      (161,194), (220,237), (205,258), (146,215)    botal tat            0.090472
7      (158,232), (231,287), (215,308), (142,253)    'Saturated           0.17